In [2]:
import os
import pandas as pd

qa_file_name = 'gpt-4o-batch-all-target_extract_gpt-4o-QAExtractor-batch_pid_0.jsonl.csv'
qa_dataset_path = os.path.join(os.path.abspath('../../.cache'), qa_file_name)
df = pd.read_csv(qa_dataset_path)

In [3]:
import json

def get_context(input_text):
    lines = input_text.split('\n')
    for idx, text in enumerate(lines):
        if text.startswith('Here is the context'):
            # The next line will be the context string
            if idx + 1 < len(lines):
                return lines[idx + 1]

def get_quetions_answers_and_groundtruth(df):
    questions = []
    ground_truths = []
    correct_contexts = []

    for row_id, obj_str in enumerate(df['objs']):
        objs = json.loads(obj_str)
        if len(objs) == 0:
            print(f"[Invalid rqa pairs] No objs found at row id: {row_id} which should have {df['qar_num'][row_id]} questions")
        for obj in objs:
            questions.append(obj['Question'])
            ground_truths.append(obj['Answer'])
            correct_contexts.append([get_context(df['input_text'][row_id])])
    return questions, ground_truths, correct_contexts

q, g, c = get_quetions_answers_and_groundtruth(df)

[Invalid rqa pairs] No objs found at row id: 10 which should have 3 questions
[Invalid rqa pairs] No objs found at row id: 126 which should have 1 questions
[Invalid rqa pairs] No objs found at row id: 195 which should have 1 questions
[Invalid rqa pairs] No objs found at row id: 415 which should have 1 questions
[Invalid rqa pairs] No objs found at row id: 457 which should have 1 questions
[Invalid rqa pairs] No objs found at row id: 556 which should have 1 questions
[Invalid rqa pairs] No objs found at row id: 561 which should have 1 questions
[Invalid rqa pairs] No objs found at row id: 574 which should have 1 questions
[Invalid rqa pairs] No objs found at row id: 579 which should have 1 questions
[Invalid rqa pairs] No objs found at row id: 631 which should have 1 questions
[Invalid rqa pairs] No objs found at row id: 641 which should have 1 questions
[Invalid rqa pairs] No objs found at row id: 709 which should have 1 questions
